In [1]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
# read dataset and set variables
df = pd.read_csv('reduced_dim_space_ddG.csv')
# We have lot of rows that have Enantiomeric Excess == 0. These ones we will exclude from the dataset.
df_cleaned = df[df['ddG (% ee)'] != 0]
X = df_cleaned[['x', 'y', 'z']]
y = df_cleaned['ddG (% ee)']

In [3]:
# Set KNN with the entire dataset
knn_train_dataset = X
# Perform k-nearest neighbors analysis
num_neighbors = 1  # Number of neighbors to consider
knn = NearestNeighbors(n_neighbors=num_neighbors)
knn.fit(knn_train_dataset)  # Using the remaining 150 catalysts for validation


NearestNeighbors(n_neighbors=1)

In [18]:
from sklearn.cross_decomposition import PLSRegression
model = PLSRegression(n_components=2)
#X_train = pd.DataFrame()
#y_train = pd.DataFrame()
def add_data_remodel(X, y):
    global X_train
    X_train = pd.concat([X_train, X])
    global y_train
    y_train = pd.concat([y_train, y])
    # fitting the model (refresh of PLS with new rows included
    model.fit(X_train, y_train)
    print("We refreshed the PLS model with {0} rows".format(len(X_train)))


In [20]:
# start with N rows
START_ROWS_NUM = 3
X_initial = X[0:START_ROWS_NUM]
y_initial = y[0:START_ROWS_NUM]

add_data_remodel(X_initial, y_initial)

We refreshed the PLS model with 12 rows
